# 컴파일(Compilation)
MLX는 계산 그래프를 컴파일하는 `compile()` 함수 변환 기능을 제공합니다. 함수 컴파일을 통해 공통 작업을 병합하고 특정 연산을 결합하여 더 작은 그래프를 생성합니다. 이는 많은 경우 실행 시간과 메모리 사용량에서 큰 개선을 가져올 수 있습니다.

`compile()`을 시작하는 것은 간단하지만, 더 복잡한 그래프와 고급 사용 사례를 위해 주의해야할 부분도 있습니다.

### 컴파일 기본
간단한 예제부터 시작해 보겠습니다:

In [1]:
import mlx.core as mx
def fun(x, y):
    return mx.exp(-x) + y

x = mx.array(1.0)
y = mx.array(2.0)

# 일반 호출, 컴파일 없음
print(fun(x, y))

# 함수를 컴파일
compiled_fun = mx.compile(fun)

print(compiled_fun(x, y))


array(2.36788, dtype=float32)
array(2.36788, dtype=float32)


컴파일된 함수를 처음 호출할 때, MLX는 계산 그래프를 구축하고, 최적화하며, 코드를 생성 및 컴파일합니다. 이는 상대적으로 느릴 수 있습니다. 그러나 MLX는 컴파일된 함수를 캐시하므로, 컴파일된 함수를 여러 번 호출해도 새로운 컴파일이 시작되지 않습니다. 따라서 여러 번 사용할 계획이 있는 함수는 일반적으로 컴파일해야 합니다.

In [2]:
def fun(x, y):
    return mx.exp(-x) + y

x = mx.array(1.0)
y = mx.array(2.0)

compiled_fun = mx.compile(fun)

# 여기서 컴파일됨
compiled_fun(x, y)

# 다시 컴파일되지 않음
compiled_fun(x, y)

# 다시 컴파일되지 않음
mx.compile(fun)(x, y)


array(2.36788, dtype=float32)

함수가 다시 컴파일될 수 있는 중요한 경우가 몇 가지 있습니다:

형태(shape)나 차원 수가 변경될 때
입력 유형이 변경될 때
함수에 대한 입력 수가 변경될 때
어떤 경우에는 컴파일 스택의 일부만 다시 실행되고 (예: 형태 변경 시), 다른 경우에는 전체 컴파일 스택이 다시 실행될 수 있습니다 (예: 유형 변경 시). 일반적으로 함수를 너무 자주 컴파일하는 것은 피해야 합니다.

다른 패턴으로는 자주 생성되고 파괴되는 함수를 컴파일하는 경우를 주의해야 합니다. 예를 들어, 루프 내에서 익명 함수를 컴파일할 때 이런 일이 발생할 수 있습니다:

In [3]:
a = mx.array(1.0)
# 이렇게 하지 마세요. 각 반복마다 람다를 컴파일합니다.
for _ in range(5):
    mx.compile(lambda x: mx.exp(mx.abs(x)))(a)


### 성능 향상 예시(중요)
mlx.nn.gelu()는 Transformer 기반 모델에서 자주 사용되는 비선형 활성화 함수입니다. 이 구현에는 여러 단항 및 이항 요소별 연산이 포함됩니다:

In [22]:
import math
def gelu(x):
    return x * (1 + mx.erf(x / math.sqrt(2))) / 2

이 함수를 작은 배열에 사용할 경우 오버헤드에 의해 제한될 것입니다. 큰 배열에 사용할 경우 메모리 대역폭에 의해 제한됩니다. 그러나 gelu의 모든 연산은 **compile()을 통해 단일 커널로 결합될 수 있습니다.** 이렇게 하면 두 경우 모두 상당한 속도 향상을 얻을 수 있습니다.

일반 함수와 컴파일된 함수의 실행 시간을 비교해 보겠습니다. 다음은 워밍업을 수행하고 동기화를 처리하는 함수입니다.

In [23]:
import time
import mlx.nn as nn
def timeit(fun, x):
    # 워밍업
    for _ in range(10):
        mx.eval(fun(x))

    tic = time.perf_counter()
    for _ in range(100):
        mx.eval(fun(x))
    toc = time.perf_counter()
    tpi = 1e3 * (toc - tic) / 100
    print(f"Iteration당 시간: {tpi:.3f} (ms)")


In [28]:
x = mx.random.uniform(shape=(32, 1000, 1024))

In [29]:
timeit(nn.gelu, x)

Iteration당 시간: 3.543 (ms)


In [30]:
compiled = mx.compile(nn.gelu)
timeit(compiled, x)

Iteration당 시간: 3.156 (ms)


참고: 최신 MLX 기준으로 CPU 함수는 완전히 컴파일되지 않습니다. CPU 함수를 컴파일하는 것도 여전히 유용할 수 있지만, GPU에서 실행되는 연산을 컴파일하는 것만큼 큰 속도 향상을 얻을 수는 없습니다.



### 디버깅
컴파일된 함수가 처음 호출될 때, 플레이스홀더 입력을 사용하여 추적됩니다. 이로 인해 컴파일된 함수 내부에서 배열을 계산/연산(예: 배열의 내용을 출력)할 수 없습니다.디버깅할 때 배열을 검사하는 것이 유용할 수 있습니다. 이를 위해 컴파일을 전역적으로 비활성화하는 disable_compile() 함수나 MLX_DISABLE_COMPILE 플래그를 사용할 수 있습니다. 예를 들어, 다음은 fun이 컴파일되었음에도 불구하고 문제가 없습니다.

In [33]:
@mx.compile
def fun(x):
    z = -x
    print(z)  # 문제 없음
    return mx.exp(z)

mx.disable_compile()
fun(mx.array(5.0))

array(-5, dtype=float32)


array(0.00673795, dtype=float32)

### 훈련 그래프와 컴파일
이 섹션에서는  `mlx.nn.Module`과 상태가 있는 `mlx.optimizers.Optimizer`를 사용하여 모델을 훈련하는 일반적인 설정의 간단한 예제와 함께 `compile()`을 사용하는 방법을 단계별로 설명합니다. `compile()`을 사용하여 전체 전진, 후진 및 업데이트를 컴파일하는 방법을 보여줍니다.

먼저, 컴파일 없이 간단한 예제를 보겠습니다:

In [38]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim

# 10개의 특징을 가진 4개의 예제
x = mx.random.uniform(shape=(4, 10))

# 0, 1 목표
y = mx.array([0, 1, 0, 1])

# 간단한 선형 모델
model = nn.Linear(10, 1)

# 모멘텀을 가진 SGD
optimizer = optim.SGD(learning_rate=0.1, momentum=0.8)

def loss_fn(model, x, y):
    logits = model(x).squeeze()
    return nn.losses.binary_cross_entropy(logits, y)

loss_and_grad_fn = nn.value_and_grad(model, loss_fn)

# 10번의 경사하강법 수행
for it in range(10):
    loss, grads = loss_and_grad_fn(model, x, y)
    optimizer.update(model, grads)
    mx.eval(model.parameters(), optimizer.state)
    print(loss)


array(0.653052, dtype=float32)
array(0.645132, dtype=float32)
array(0.631564, dtype=float32)
array(0.614461, dtype=float32)
array(0.59548, dtype=float32)
array(0.57572, dtype=float32)
array(0.555785, dtype=float32)
array(0.535937, dtype=float32)
array(0.516271, dtype=float32)
array(0.496845, dtype=float32)


업데이트를 컴파일하려면 모든 것을 함수에 넣고 적절한 입력 및 출력 캡처와 함께 컴파일할 수 있습니다. 다음은 컴파일된 동일한 예제입니다:

In [37]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
from functools import partial

# 10개의 특징을 가진 4개의 예제
x = mx.random.uniform(shape=(4, 10))

# 0, 1 목표
y = mx.array([0, 1, 0, 1])

# 간단한 선형 모델
model = nn.Linear(10, 1)

# 모멘텀을 가진 SGD
optimizer = optim.SGD(learning_rate=0.1, momentum=0.8)

def loss_fn(model, x, y):
    logits = model(x).squeeze()
    return nn.losses.binary_cross_entropy(logits, y)

# 입력 및 출력으로 캡처할 상태
state = [model.state, optimizer.state]

@partial(mx.compile, inputs=state, outputs=state) # 여기서 컴파일
def step(x, y):
    loss_and_grad_fn = nn.value_and_grad(model, loss_fn)
    loss, grads = loss_and_grad_fn(model, x, y)
    optimizer.update(model, grads)
    return loss

# 10번의 경사하강법 수행
for it in range(10):
    loss = step(x, y)
    # 모델 및 최적화기 상태 평가
    mx.eval(state)
    print(loss)


array(0.64857, dtype=float32)
array(0.644333, dtype=float32)
array(0.637261, dtype=float32)
array(0.628669, dtype=float32)
array(0.619493, dtype=float32)
array(0.61022, dtype=float32)
array(0.600976, dtype=float32)
array(0.591691, dtype=float32)
array(0.58225, dtype=float32)
array(0.5726, dtype=float32)


참고:  `mlx.nn.Dropout()`과 같은 랜덤 샘플링을 수행하는 모듈을 사용하는 경우, 컴파일에서 `mx.random.state`도 포함해야 합니다. 즉, `state = [model.state, optimizer.state, mx.random.state]`과 같이 설정해야 합니다.

### 마치며...
이번 챕터를 요약하자면, 일반 함수와 컴파일된 함수의 차이는 코드 실행의 최적화 수준에 있습니다. 일반 함수는 직관적이지만 덜 최적화된 방식으로 실행되고, 컴파일된 함수는 성능을 최적화하기 위해 연산을 병합하거나 메모리를 효율적으로 사용하는 등 다양한 최적화를 적용하여 실행됩니다.

참조 :https://ml-explore.github.io/mlx/build/html/usage/compile.html